In [ ]:
from IPython.core.getipython import get_ipython
from matplotlib import pyplot as plt
import sys
import h5py
import os
import pandas as pd
import seaborn as sns
sys.path.append("..")
import json
import matplotlib.patches as mpatches
try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass
import datadoc_util
from labrotation import file_handling as fh
sns.set(font_scale=3)
sns.set_style("whitegrid")

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
output_folder = env_dict["DOWNLOADS_FOLDER"]
print(f"Output files will be saved to {output_folder}")

In [ ]:
# assumption: inside the documentation folder, the subfolders carry the id of each mouse (not exact necessarily, but they 
# can be identified by the name of the subfolder). 
# Inside the subfolder xy (for mouse xy), xy_grouping.xlsx and xy_segmentation.xlsx can be found.
# xy_grouping.xlsx serves the purpose of finding the recordings belonging together, and has columns:
# folder, nd2, labview, lfp, face_cam_last, nikon_meta, experiment_type, day
# xy_segmentation.xlsx contains frame-by-frame (given by a set of disjoint intervals forming a cover for the whole recording) 
# classification of the events in the recording ("normal", seizure ("sz"), sd wave ("sd_wave") etc.). The columns:
# folder, interval_type, frame_begin, frame_end.

# TODO: write documentation on contents of xlsx files (what the columns are etc.)
if "DATA_DOCU_FOLDER" in env_dict.keys():
    docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    docu_folder = fh.open_dir("Choose folder containing folders for each mouse!")
print(f"Selected folder:\n\t{docu_folder}")

In [ ]:
ddoc = datadoc_util.DataDocumentation(docu_folder)
ddoc.loadDataDoc()

In [ ]:
fpath = fh.open_file(f"Open json file containing all files to open")

In [ ]:
with open(fpath, "r") as f:
    dict_fpaths = json.load(f)

In [ ]:
ddoc.getColorForMouseId("WEZ-8924")

In [ ]:
dict_n_cells = dict()  # {mouse_id:  exp_type: {"pre": [n_cells for each recording], "post": [n_cells for each recording], "uuid": [corresponding uuids]}}}
dict_mouse_colors = dict() # {mouse_id: color}

for mouse_id in dict_fpaths:
    dict_n_cells[mouse_id] = dict()
    for exp_type in dict_fpaths[mouse_id]:
        dict_n_cells[mouse_id][exp_type] = dict()
        fpaths_pre = dict_fpaths[mouse_id][exp_type]["pre"]
        fpaths_post = dict_fpaths[mouse_id][exp_type]["post"]
        assert len(fpaths_pre) == len(fpaths_post)
        n_cells_pre = []
        n_cells_post = []
        uuids = []
        for i_recording in range(len(fpaths_pre)):
            with h5py.File(fpaths_pre[i_recording], "r") as hf:
                n_cells_pre.append(hf["estimates"]["A"]["shape"][1])
                uuids.append(hf.attrs["uuid"])
            with h5py.File(fpaths_post[i_recording], "r") as hf:
                n_cells_post.append(hf["estimates"]["A"]["shape"][1])
        dict_n_cells[mouse_id][exp_type]["pre"] = n_cells_pre
        dict_n_cells[mouse_id][exp_type]["post"] = n_cells_post
        dict_n_cells[mouse_id][exp_type]["uuid"] = uuids
        dict_mouse_colors[mouse_id] = ddoc.getColorForMouseId(mouse_id)



In [ ]:
analysis_type = "tmev" if "tmev" in dict_n_cells[list(dict_n_cells.keys())[0]] else "chr2"

In [ ]:
fig = plt.figure(figsize=(6,6))

dict_colors = {"ctl": "green", "sd": "red", "szsd": "blue", "tmev": "orange"}

patch_sd = mpatches.Patch(color=dict_colors["sd"], label='SD stimulation (488nm)')
patch_szsd = mpatches.Patch(color=dict_colors["szsd"], label='Sz+SD stimulation (488nm)')
patch_ctl = mpatches.Patch(color=dict_colors["ctl"], label='control (561nm)')
patch_tmev = mpatches.Patch(color=dict_colors["tmev"], label='TMEV')


for i, mouse_id in enumerate(dict_n_cells):
    for exp_type in dict_n_cells[mouse_id].keys():
        color = dict_colors[exp_type]
        n_cells_pre = dict_n_cells[mouse_id][exp_type]["pre"]
        n_cells_post = dict_n_cells[mouse_id][exp_type]["post"]
        for i_exp in range(len(n_cells_pre)):
            plt.plot(["pre", "post"], [n_cells_pre[i_exp], n_cells_post[i_exp]], color=color)

if analysis_type == "tmev":
    plt.legend(handles=[patch_tmev], fontsize=12)
else:
    plt.legend(handles=[patch_sd, patch_szsd, patch_ctl], fontsize=12)
plt.ylabel("# cells detected")
plt.tight_layout()
plt.savefig(os.path.join(output_folder, f"cell_count_pre_post_{analysis_type}.pdf"))
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))

dict_colors = {"ctl": "green", "sd": "red", "szsd": "blue", "tmev": "orange"}

patch_sd = mpatches.Patch(color=dict_colors["sd"], label='SD stimulation (488nm)')
patch_szsd = mpatches.Patch(color=dict_colors["szsd"], label='Sz+SD stimulation (488nm)')
patch_ctl = mpatches.Patch(color=dict_colors["ctl"], label='control (561nm)')
patch_tmev = mpatches.Patch(color=dict_colors["tmev"], label='TMEV')

for i, mouse_id in enumerate(dict_n_cells):
    for exp_type in dict_n_cells[mouse_id].keys():
        color = dict_colors[exp_type]
        n_cells_pre = dict_n_cells[mouse_id][exp_type]["pre"]
        n_cells_post = dict_n_cells[mouse_id][exp_type]["post"]
        for i_exp in range(len(n_cells_pre)):
            plt.plot(["pre", "post"], [100.*n_cells_pre[i_exp]/n_cells_pre[i_exp], 100.*n_cells_post[i_exp]/n_cells_pre[i_exp]], color=color)

if analysis_type == "tmev":
    plt.legend(handles=[patch_tmev], fontsize=12)
else:
    plt.legend(handles=[patch_sd, patch_szsd, patch_ctl], fontsize=12)
plt.ylabel("% of baseline cells detected", fontsize=16)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, f"cell_count_pre_post_%_{analysis_type}.pdf"))
plt.show()

In [ ]:
col_mouse = []  # mouse id
col_exp_type = []  # szsd, sd, ctl
col_uuid = []  # unique for recording
col_cell_count_pre = []
col_cell_count_post = []
col_colors = []

for i, mouse_id in enumerate(dict_n_cells):
    for exp_type in dict_n_cells[mouse_id].keys():
        color = dict_mouse_colors[mouse_id]
        n_cells_pre = dict_n_cells[mouse_id][exp_type]["pre"]
        n_cells_post = dict_n_cells[mouse_id][exp_type]["post"]
        uuids = dict_n_cells[mouse_id][exp_type]["uuid"]
        mouse_ids = [mouse_id]*len(n_cells_pre)
        exp_types = [exp_type]*len(n_cells_pre)
        colors = [color]*len(n_cells_pre)

        col_mouse.extend(mouse_ids)
        col_exp_type.extend(exp_types)
        col_uuid.extend(uuids)
        col_cell_count_pre.extend(n_cells_pre)
        col_cell_count_post.extend(n_cells_post)
        col_colors.extend(colors)

df = pd.DataFrame({"mouse_id": col_mouse, "exp_type": col_exp_type, "uuid": col_uuid, "cell_count_pre": col_cell_count_pre, "cell_count_post": col_cell_count_post, "color": col_colors})
df = df.sort_values(by=["mouse_id", "exp_type", "uuid"])

In [ ]:
df["post_pre_ratio"] = df["cell_count_post"]/df["cell_count_pre"]

In [ ]:
df.to_excel(os.path.join(output_folder, f"cell_count_pre_post_{analysis_type}.xlsx"), index=False)

In [ ]:

fig = plt.figure(figsize=(10,10))
sns.boxplot(x="exp_type", y="post_pre_ratio", hue="exp_type", data=df, palette=dict_colors)
plt.ylabel("post/pre count ratio")
plt.xlabel("experiment type")
plt.tight_layout()
plt.savefig(os.path.join(output_folder, f"cell_count_pre_post_boxplot_{analysis_type}.pdf"))
plt.show()